In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import requests
import json
from openai import OpenAI
import configparser
import argparse

import tools.ProcessFiles as pf

In [ ]:
# pf.get_filelist(1, data_path='/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FIN_workbook')

FINAL_DATA_PATH = '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FIN_workbook'
json_files = []
for root, _, files in os.walk(FINAL_DATA_PATH):
    for f in files:
        if f.endswith(".json") and ('merged' not in f):
        #  and ('Lv5' not in root):
            json_files.append(os.path.join(root, f))

In [ ]:
len(json_files)

In [ ]:
multiple, short, essay = [],[],[]

for file in json_files: 
    origin = json.load(open(file, 'r', encoding='utf-8'))
    for qna in origin:
        if len(qna['qna_data']['description']['answer']) > 0:
            if qna.get('qna_type') == "multiple-choice":
                multiple.append(qna)
            elif qna.get('qna_type') == "short-answer":
                short.append(qna)
            elif qna.get('qna_type') == "essay":
                essay.append(qna)
# data_dir = os.path.join(base_dir,'Lv2', 're')
# origin = json.load(open(os.path.join(base_dir, 'merged_qna_set.json'), 'r', encoding='utf-8'))

In [ ]:
len(multiple), len(short), len(essay)

In [ ]:
import random
random_multiple = random.sample(multiple, 100)

from tools.ProcessQnA import replace_tags_in_qna_data

for qna in random_multiple:
    if len(qna.get('additional_tags_found', [])) > 0:
        print("원본", qna.get('qna_data'))
        result_qna_data = replace_tags_in_qna_data(qna.get('qna_data'), qna.get('additional_tag_data', []))
        # print("처리 중", result_qna_data)
        qna['qna_data'] = result_qna_data
        print("처리된", qna.get('qna_data'))
        print("--------------------------------")

In [ ]:
import pandas as pd
import numpy as np
import re
import time
import logging
from typing import List, Dict, Tuple, Iterable, Set
from dataclasses import dataclass
from tqdm import tqdm

# -----------------------------
# 0) 로깅 설정
# -----------------------------
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('evaluation.log', encoding='utf-8')
    ]
)
logger = logging.getLogger(__name__)

# -----------------------------
# 1) 유틸: 텍스트 정규화
# -----------------------------
CIRCLED_MAP = {"①":"1","②":"2","③":"3","④":"4","⑤":"5"}

def normalize_option_text(s: str) -> str:
    """선지 앞에 붙은 ①~⑤, 1), (1), 1. 등 번호 표기를 제거하고 본문만 남김."""
    if s is None:
        return ""
    s = str(s).strip()
    # ①~⑤ 제거
    s = re.sub(r"^\s*[①-⑤]\s*", "", s)
    # 1), (1), 1. 등 제거
    s = re.sub(r"^\s*(?:\(?([1-5])\)?[.)])\s*", "", s)
    return s.strip()

def parse_answer_set(ans: str) -> Set[int]:
    """'①, ⑤' 같은 복수정답도 {1,5}로 파싱. 빈/이상값은 빈 set."""
    if not ans:
        return set()
    s = str(ans)
    # ①~⑤ 를 1~5로 치환
    for k, v in CIRCLED_MAP.items():
        s = s.replace(k, v)
    # 쉼표/슬래시/공백 구분 모두 허용하여 1~5 추출
    nums = re.findall(r"[1-5]", s)
    return set(int(n) for n in nums)

# -----------------------------
# 2) JSON → df_all 변환
# -----------------------------
def json_to_df_all(json_list: List[dict]) -> pd.DataFrame:
    """
    입력 JSON(list[dict])을 파싱해 df_all 생성.
    컬럼: book_id, tag, id, question, opt1..opt5, answer_set
    """
    rows = []
    for item in json_list:
        book_id = str(item.get("file_id", ""))
        qna = item.get("qna_data", {}) or {}
        tag  = qna.get("tag", "")
        desc = qna.get("description", {}) or {}
        q    = (desc.get("question") or "").strip()
        opts = desc.get("options") or []
        # 5지선다 기준으로 빈칸 보정
        opts = list(opts)[:5] + [""] * max(0, 5 - len(opts))
        opts = [normalize_option_text(x) for x in opts]
        ans_set = parse_answer_set(desc.get("answer", ""))

        rows.append({
            "book_id": book_id,
            "tag": tag,
            "id": f"{book_id}_{tag}",
            "question": q,
            "opt1": opts[0], "opt2": opts[1], "opt3": opts[2], "opt4": opts[3], "opt5": opts[4],
            "answer_set": ans_set
        })
    df = pd.DataFrame(rows)
    # 혹시 id 중복이 있으면 마지막 것 유지(필요시 정책 변경)
    df = df.drop_duplicates("id", keep="last").reset_index(drop=True)
    return df

# -----------------------------
# 3) 배치 사용자 프롬프트 생성 (50문제)
# -----------------------------
SYSTEM_PROMPT = """당신은 금융전문가이자 객관식 문제 풀이 전문가입니다.
여러 금융 객관식 문제에 대해, 각 문제의 정답 "번호만" 하나 선택합니다.

규칙
- 각 문제는 고유 ID와 함께 제시됩니다.
- 출력은 반드시 한 줄당 "ID<TAB>번호" 형식으로만 합니다. (예: SS0000_q_0377_0001<TAB>3)
- 다른 글자, 마크다운, 이유, 기호는 절대 출력하지 않습니다.
- 모든 문제는 보기(1~5) 중 하나만 고릅니다.
- 출력 줄 수는 입력 문제 개수와 동일해야 합니다.
"""

def build_user_prompt(batch_df: pd.DataFrame) -> str:
    lines = []
    lines.append("다음은 금융 객관식 문제들입니다. 각 문제에 대해 정답 번호만 고르세요.\n")
    lines.append("문제들")
    for _, r in batch_df.iterrows():
        lines.append(f"ID: {r['id']}")
        lines.append(f"Q: {r['question']}")
        lines.append(f"1) {r['opt1']}")
        lines.append(f"2) {r['opt2']}")
        lines.append(f"3) {r['opt3']}")
        lines.append(f"4) {r['opt4']}")
        lines.append(f"5) {r['opt5']}\n")
    lines.append("출력 형식(중요)")
    for _, r in batch_df.iterrows():
        lines.append(f"{r['id']}\\t{{번호}}")
    return "\n".join(lines)

# -----------------------------
# 4) LLM 호출 추상화 (모의/실제) - 개선된 버전
# -----------------------------
def call_llm(model_name: str, system_prompt: str, user_prompt: str, mock_mode: bool=False, max_retries: int=3) -> str:
    """
    - mock_mode=True면 임의 번호(1~5)를 생성해 파이프라인 검증용 출력 반환.
    - 실제 환경에서는 이 함수를 OpenAI/사내엔진 호출로 교체.
    - 에러 핸들링 및 재시도 로직 포함
    """
    if mock_mode:
        logger.info(f"[MOCK] 모델 {model_name} 호출 시작")
        # 입력 user_prompt에서 ID 목록 회수
        ids = [ln.split("\t")[0] for ln in user_prompt.splitlines() if "\t{번호}" in ln]
        # 무작위 예측(1~5)
        rng = np.random.default_rng(42)
        preds = rng.integers(1, 6, size=len(ids))
        result = "\n".join(f"{_id}\t{int(a)}" for _id, a in zip(ids, preds))
        logger.info(f"[MOCK] 모델 {model_name} 호출 완료 - {len(ids)}개 문제 처리")
        return result
    
    else:
        import tools.Openrouter as Openrouter
        
        for attempt in range(max_retries):
            try:
                logger.info(f"[API] 모델 {model_name} 호출 시작 (시도 {attempt + 1}/{max_retries})")
                start_time = time.time()
                
                ans = Openrouter.query_model_openrouter(system_prompt, user_prompt, model_name)
                
                elapsed_time = time.time() - start_time
                logger.info(f"[API] 모델 {model_name} 호출 완료 - 소요시간: {elapsed_time:.2f}초")
                
                return ans
                
            except Exception as e:
                logger.warning(f"[API] 모델 {model_name} 호출 실패 (시도 {attempt + 1}/{max_retries}): {str(e)}")
                if attempt == max_retries - 1:
                    logger.error(f"[API] 모델 {model_name} 최종 실패 - 모든 재시도 소진")
                    raise e
                else:
                    wait_time = 2 ** attempt  # 지수 백오프
                    logger.info(f"[API] {wait_time}초 후 재시도...")
                    time.sleep(wait_time)

# -----------------------------
# 5) 모델 출력 파싱 (ID<TAB>번호)
# -----------------------------
def parse_model_output(raw: str, expected_ids: List[str]) -> Dict[str, float]:
    """
    모델 원시 출력(raw)을 {id: answer(1~5)}로 변환.
    - 'ID\\t번호' 포맷 기준
    - 잘못된 줄/누락 줄은 NaN 처리
    """
    id_set = set(expected_ids)
    out: Dict[str, float] = {k: np.nan for k in expected_ids}

    for ln in (raw or "").splitlines():
        ln = ln.strip()
        if not ln or "\t" not in ln:
            continue
        left, right = ln.split("\t", 1)
        _id = left.strip()
        if _id not in id_set:
            continue
        # 첫 번째 1~5 추출
        m = re.search(r"[1-5]", right)
        if m:
            out[_id] = float(m.group(0))
    return out

# -----------------------------
# 6) 파이프라인: n개 샘플 → 50개씩 × 모델 호출 → DF 정리/정확도 (개선된 버전)
# -----------------------------
def run_eval_pipeline(
    json_list: List[dict],
    models: List[str],
    sample_size: int = 300,
    batch_size: int = 50,
    seed: int = 42,
    mock_mode: bool = False,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    반환:
      df_all      : 전체 원장 (정규화 선지 + answer_set)
      pred_long   : (id, model_name, answer) 롱 포맷
      pred_wide   : id 기준 모델별 예측 와이드
      acc_by_model: 모델별 정확도 (복수정답 지원: 예측 ∈ answer_set 이면 정답)
    """
    logger.info(f"평가 파이프라인 시작 - 샘플수: {sample_size}, 배치크기: {batch_size}, 모델수: {len(models)}")
    
    # (1) JSON → df_all
    logger.info("1단계: JSON 데이터를 DataFrame으로 변환 중...")
    df_all = json_to_df_all(json_list)
    df_all = df_all.sort_values(by=['book_id', 'tag'], ascending=False).reset_index(drop=True)
    logger.info(f"전체 데이터: {len(df_all)}개 문제")

    # (2) 샘플링
    logger.info(f"2단계: {sample_size}개 샘플 추출 중...")
    df_sample = df_all.sample(n=sample_size, random_state=seed).reset_index(drop=True)
    logger.info(f"샘플 데이터: {len(df_sample)}개 문제")

    # (3) 배치 분할
    batches = [df_sample.iloc[i:i+batch_size] for i in range(0, len(df_sample), batch_size)]
    logger.info(f"3단계: {len(batches)}개 배치로 분할 완료")

    # (4) 모델 호출/파싱 누적
    logger.info("4단계: 모델 호출 및 예측 시작...")
    rows = []
    total_calls = len(batches) * len(models)
    
    # 전체 진행상황 표시
    with tqdm(total=total_calls, desc="모델 호출 진행", unit="call") as pbar:
        for bidx, bdf in enumerate(batches, 1):
            user_prompt = build_user_prompt(bdf)
            ids = bdf["id"].tolist()
            
            for model in models:
                try:
                    # 배치별 진행상황 표시
                    pbar.set_description(f"배치 {bidx}/{len(batches)} - {model}")
                    
                    raw = call_llm(model, SYSTEM_PROMPT, user_prompt, mock_mode=mock_mode)
                    parsed = parse_model_output(raw, ids)
                    
                    # 파싱 결과 검증
                    valid_predictions = sum(1 for v in parsed.values() if not np.isnan(v))
                    logger.info(f"배치 {bidx} - {model}: {valid_predictions}/{len(ids)}개 유효 예측")
                    
                    for _id in ids:
                        rows.append({"id": _id, "model_name": model, "answer": parsed[_id]})
                    
                    pbar.update(1)
                    
                except Exception as e:
                    logger.error(f"배치 {bidx} - {model} 처리 중 오류: {str(e)}")
                    # 오류 발생 시 NaN으로 채움
                    for _id in ids:
                        rows.append({"id": _id, "model_name": model, "answer": np.nan})
                    pbar.update(1)

    logger.info("5단계: 결과 데이터 정리 중...")
    pred_long = pd.DataFrame(rows)
    pred_long = pred_long.sort_values(by=['id'], ascending=True).reset_index(drop=True)
    
    # (5) 와이드 포맷
    pred_wide = pred_long.pivot(index="id", columns="model_name", values="answer").reset_index()
    pred_wide = pred_wide.sort_values(by=['id'], ascending=True).reset_index(drop=True)

    # (6) 정확도 계산
    logger.info("6단계: 정확도 계산 중...")
    key = df_sample[["id", "answer_set"]].copy()
    
    def _is_correct(pred: float, s: Set[int]) -> float:
        if np.isnan(pred) or not s:
            return np.nan
        return float(int(pred) in s)

    merged = pred_long.merge(key, on="id", how="left")
    merged["correct"] = merged.apply(lambda r: _is_correct(r["answer"], r["answer_set"]), axis=1)

    acc_by_model = (
        merged.groupby("model_name", dropna=False)["correct"]
        .mean()
        .reset_index()
        .rename(columns={"correct": "accuracy"})
        .sort_values("accuracy", ascending=False)
    )
    
    # 결과 요약 로깅
    logger.info("평가 완료!")
    logger.info("모델별 정확도:")
    for _, row in acc_by_model.iterrows():
        logger.info(f"  {row['model_name']}: {row['accuracy']:.3f}")
    
    return df_all, pred_long, pred_wide, acc_by_model

In [ ]:
# -----------------------------
# O, X 문제 처리 개선
# -----------------------------

def is_ox_question(question: str, options: list) -> bool:
    """O, X 문제인지 판단"""
    if not options or len(options) == 0:
        return False
    # options가 비어있거나 2개 이하이고, O/X 형태인지 확인
    if len(options) <= 2:
        option_text = " ".join(options).upper()
        return "O" in option_text or "X" in option_text
    return False

def parse_answer_set_improved(ans: str, question: str = "", options: list = None) -> Set[int]:
    """개선된 정답 파싱 함수 - O, X 문제도 처리"""
    if not ans:
        return set()
    s = str(ans).strip()
    
    # O, X 문제 처리
    if s.upper() in ['O', 'X']:
        # O, X 문제는 1번(O), 2번(X)으로 변환
        return {1} if s.upper() == 'O' else {2}
    
    # ①~⑤ 를 1~5로 치환
    for k, v in CIRCLED_MAP.items():
        s = s.replace(k, v)
    # 쉼표/슬래시/공백 구분 모두 허용하여 1~5 추출
    nums = re.findall(r"[1-5]", s)
    return set(int(n) for n in nums)

def json_to_df_all_improved(json_list: List[dict]) -> pd.DataFrame:
    """
    개선된 JSON → df_all 변환 함수 - O, X 문제도 처리
    컬럼: book_id, tag, id, question, opt1..opt5, answer_set, is_ox_question
    """
    rows = []
    for item in json_list:
        book_id = str(item.get("file_id", ""))
        qna = item.get("qna_data", {}) or {}
        tag  = qna.get("tag", "")
        desc = qna.get("description", {}) or {}
        q    = (desc.get("question") or "").strip()
        opts = desc.get("options") or []
        
        # O, X 문제인지 판단
        is_ox = is_ox_question(q, opts)
        
        if is_ox:
            # O, X 문제는 2개 선지로 고정
            opts = ["O", "X"] + [""] * 3
        else:
            # 5지선다 기준으로 빈칸 보정
            opts = list(opts)[:5] + [""] * max(0, 5 - len(opts))
        
        opts = [normalize_option_text(x) for x in opts]
        ans_set = parse_answer_set_improved(desc.get("answer", ""), q, opts)

        rows.append({
            "book_id": book_id,
            "tag": tag,
            "id": f"{book_id}_{tag}",
            "question": q,
            "opt1": opts[0], "opt2": opts[1], "opt3": opts[2], "opt4": opts[3], "opt5": opts[4],
            "answer_set": ans_set,
            "is_ox_question": is_ox
        })
    df = pd.DataFrame(rows)
    # 혹시 id 중복이 있으면 마지막 것 유지(필요시 정책 변경)
    df = df.drop_duplicates("id", keep="last").reset_index(drop=True)
    return df

# O, X 문제 통계 확인 함수
def analyze_ox_questions(df: pd.DataFrame):
    """O, X 문제 분석"""
    ox_questions = df[df['is_ox_question'] == True]
    regular_questions = df[df['is_ox_question'] == False]
    
    print(f"📊 문제 유형 분석")
    print(f"   - O, X 문제: {len(ox_questions)}개")
    print(f"   - 일반 객관식: {len(regular_questions)}개")
    print(f"   - 전체: {len(df)}개")
    
    if len(ox_questions) > 0:
        print(f"\n🔍 O, X 문제 정답 분포:")
        ox_answers = ox_questions['answer_set'].apply(lambda x: list(x) if x else [])
        answer_counts = {}
        for answers in ox_answers:
            for ans in answers:
                answer_counts[ans] = answer_counts.get(ans, 0) + 1
        
        for ans, count in sorted(answer_counts.items()):
            answer_text = "O" if ans == 1 else "X" if ans == 2 else str(ans)
            print(f"   - {answer_text}: {count}개")
    
    return ox_questions, regular_questions


In [ ]:
# -----------------------------
# 개선된 평가 파이프라인 (O, X 문제 지원)
# -----------------------------

def run_eval_pipeline_improved(
    json_list: List[dict],
    models: List[str],
    sample_size: int = 300,
    batch_size: int = 50,
    seed: int = 42,
    mock_mode: bool = False,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    O, X 문제를 지원하는 개선된 평가 파이프라인
    반환:
      df_all      : 전체 원장 (정규화 선지 + answer_set + is_ox_question)
      pred_long   : (id, model_name, answer) 롱 포맷
      pred_wide   : id 기준 모델별 예측 와이드
      acc_by_model: 모델별 정확도 (복수정답 지원: 예측 ∈ answer_set 이면 정답)
    """
    logger.info(f"개선된 평가 파이프라인 시작 - 샘플수: {sample_size}, 배치크기: {batch_size}, 모델수: {len(models)}")
    
    # (1) JSON → df_all (O, X 문제 지원)
    logger.info("1단계: JSON 데이터를 DataFrame으로 변환 중...")
    df_all = json_to_df_all_improved(json_list)
    df_all = df_all.sort_values(by=['book_id', 'tag'], ascending=False).reset_index(drop=True)
    logger.info(f"전체 데이터: {len(df_all)}개 문제")

    # O, X 문제 분석
    ox_questions, regular_questions = analyze_ox_questions(df_all)

    # (2) 샘플링
    logger.info(f"2단계: {sample_size}개 샘플 추출 중...")
    df_sample = df_all.sample(n=sample_size, random_state=seed).reset_index(drop=True)
    logger.info(f"샘플 데이터: {len(df_sample)}개 문제")

    # 샘플에서 O, X 문제 비율 확인
    sample_ox = df_sample[df_sample['is_ox_question'] == True]
    sample_regular = df_sample[df_sample['is_ox_question'] == False]
    logger.info(f"샘플 내 O, X 문제: {len(sample_ox)}개, 일반 객관식: {len(sample_regular)}개")

    # (3) 배치 분할
    batches = [df_sample.iloc[i:i+batch_size] for i in range(0, len(df_sample), batch_size)]
    logger.info(f"3단계: {len(batches)}개 배치로 분할 완료")

    # (4) 모델 호출/파싱 누적
    logger.info("4단계: 모델 호출 및 예측 시작...")
    rows = []
    total_calls = len(batches) * len(models)
    
    # 전체 진행상황 표시
    with tqdm(total=total_calls, desc="모델 호출 진행", unit="call") as pbar:
        for bidx, bdf in enumerate(batches, 1):
            user_prompt = build_user_prompt(bdf)
            ids = bdf["id"].tolist()
            
            for model in models:
                try:
                    # 배치별 진행상황 표시
                    pbar.set_description(f"배치 {bidx}/{len(batches)} - {model}")
                    
                    raw = call_llm(model, SYSTEM_PROMPT, user_prompt, mock_mode=mock_mode)
                    parsed = parse_model_output(raw, ids)
                    
                    # 파싱 결과 검증
                    valid_predictions = sum(1 for v in parsed.values() if not np.isnan(v))
                    logger.info(f"배치 {bidx} - {model}: {valid_predictions}/{len(ids)}개 유효 예측")
                    
                    for _id in ids:
                        rows.append({"id": _id, "model_name": model, "answer": parsed[_id]})
                    
                    pbar.update(1)
                    
                except Exception as e:
                    logger.error(f"배치 {bidx} - {model} 처리 중 오류: {str(e)}")
                    # 오류 발생 시 NaN으로 채움
                    for _id in ids:
                        rows.append({"id": _id, "model_name": model, "answer": np.nan})
                    pbar.update(1)

    logger.info("5단계: 결과 데이터 정리 중...")
    pred_long = pd.DataFrame(rows)
    pred_long = pred_long.sort_values(by=['id'], ascending=True).reset_index(drop=True)
    
    # (5) 와이드 포맷
    pred_wide = pred_long.pivot(index="id", columns="model_name", values="answer").reset_index()
    pred_wide = pred_wide.sort_values(by=['id'], ascending=True).reset_index(drop=True)

    # (6) 정확도 계산
    logger.info("6단계: 정확도 계산 중...")
    key = df_sample[["id", "answer_set"]].copy()
    
    def _is_correct(pred: float, s: Set[int]) -> float:
        if np.isnan(pred) or not s:
            return np.nan
        return float(int(pred) in s)

    merged = pred_long.merge(key, on="id", how="left")
    merged["correct"] = merged.apply(lambda r: _is_correct(r["answer"], r["answer_set"]), axis=1)

    acc_by_model = (
        merged.groupby("model_name", dropna=False)["correct"]
        .mean()
        .reset_index()
        .rename(columns={"correct": "accuracy"})
        .sort_values("accuracy", ascending=False)
    )
    
    # O, X 문제와 일반 문제별 정확도 분석
    sample_with_type = df_sample[["id", "is_ox_question"]].copy()
    merged_with_type = merged.merge(sample_with_type, on="id", how="left")
    
    # O, X 문제 정확도
    ox_accuracy = merged_with_type[merged_with_type['is_ox_question'] == True].groupby("model_name")["correct"].mean()
    regular_accuracy = merged_with_type[merged_with_type['is_ox_question'] == False].groupby("model_name")["correct"].mean()
    
    logger.info("평가 완료!")
    logger.info("모델별 전체 정확도:")
    for _, row in acc_by_model.iterrows():
        logger.info(f"  {row['model_name']}: {row['accuracy']:.3f}")
    
    if len(ox_accuracy) > 0:
        logger.info("O, X 문제 정확도:")
        for model, acc in ox_accuracy.items():
            logger.info(f"  {model}: {acc:.3f}")
    
    return df_all, pred_long, pred_wide, acc_by_model


In [ ]:
# -----------------------------
# O, X 문제 테스트 및 검증
# -----------------------------

# O, X 문제가 있는지 확인
print("🔍 O, X 문제 확인 중...")
df_test = json_to_df_all_improved(random_multiple)
ox_questions, regular_questions = analyze_ox_questions(df_test)

# O, X 문제가 있다면 샘플 출력
if len(ox_questions) > 0:
    print(f"\n📝 O, X 문제 샘플 (최대 3개):")
    for i, (_, row) in enumerate(ox_questions.head(3).iterrows()):
        print(f"\n{i+1}. ID: {row['id']}")
        print(f"   문제: {row['question']}")
        print(f"   선지: 1) {row['opt1']} 2) {row['opt2']}")
        print(f"   정답: {row['answer_set']} ({'O' if 1 in row['answer_set'] else 'X' if 2 in row['answer_set'] else 'Unknown'})")
else:
    print("❌ O, X 문제를 찾을 수 없습니다.")

# Mock 모드로 O, X 문제 포함 테스트
print(f"\n🧪 Mock 모드로 O, X 문제 포함 테스트 실행...")
test_data = random_multiple[:20]  # 20개 테스트
test_models = ['test-model']

try:
    df_all_test, pred_long_test, pred_wide_test, acc_test = run_eval_pipeline_improved(
        test_data, test_models, sample_size=20, batch_size=5, seed=42, mock_mode=True
    )
    
    print("✅ Mock 테스트 성공!")
    print(f"   - 처리된 문제 수: {len(df_all_test)}")
    print(f"   - O, X 문제 수: {len(df_all_test[df_all_test['is_ox_question'] == True])}")
    print(f"   - 일반 객관식 수: {len(df_all_test[df_all_test['is_ox_question'] == False])}")
    print(f"   - 예측 결과 수: {len(pred_long_test)}")
    print(f"   - 정확도: {acc_test.iloc[0]['accuracy']:.3f}")
    
except Exception as e:
    print(f"❌ Mock 테스트 실패: {str(e)}")
    logger.error(f"Mock 테스트 실패: {str(e)}")

print("\n" + "="*60)
print("🚀 O, X 문제를 포함한 실제 평가를 실행하려면 아래 셀을 실행하세요!")
print("="*60)


In [ ]:
# -----------------------------
# O, X 문제를 포함한 실제 평가 실행
# -----------------------------

# 데이터 준비
data = random_multiple
models = ['anthropic/claude-sonnet-4.5']
# , 'google/gemini-2.5-flash', 'openai/gpt-5']  # 실제 모델명

# O, X 문제를 포함한 평가 실행
logger.info("=" * 60)
logger.info("O, X 문제를 포함한 평가 시작")
logger.info("=" * 60)

# 데이터 품질 검사
df_all_temp = json_to_df_all_improved(data)
quality_issues = check_data_quality(df_all_temp, df_all_temp.sample(n=min(50, len(df_all_temp)), random_state=42))

# O, X 문제 분석
ox_questions, regular_questions = analyze_ox_questions(df_all_temp)

# 평가 실행 (O, X 문제 지원)
df_all, pred_long, pred_wide, acc = run_eval_pipeline_improved(
    data, models, sample_size=50, batch_size=10, seed=42, mock_mode=False
)

# 상세 결과 출력
print_evaluation_summary(acc, pred_long)

# O, X 문제별 정확도 분석
sample_with_type = df_all[df_all['id'].isin(pred_long['id'])][['id', 'is_ox_question']].copy()
merged_with_type = pred_long.merge(sample_with_type, on='id', how='left')

if len(merged_with_type[merged_with_type['is_ox_question'] == True]) > 0:
    print("\n" + "="*60)
    print("📊 O, X 문제 vs 일반 객관식 정확도 비교")
    print("="*60)
    
    # O, X 문제 정확도
    ox_correct = merged_with_type[merged_with_type['is_ox_question'] == True]['answer'].notna().sum()
    ox_total = len(merged_with_type[merged_with_type['is_ox_question'] == True])
    ox_acc = ox_correct / ox_total if ox_total > 0 else 0
    
    # 일반 객관식 정확도
    regular_correct = merged_with_type[merged_with_type['is_ox_question'] == False]['answer'].notna().sum()
    regular_total = len(merged_with_type[merged_with_type['is_ox_question'] == False])
    regular_acc = regular_correct / regular_total if regular_total > 0 else 0
    
    print(f"O, X 문제: {ox_correct}/{ox_total} ({ox_acc:.1%})")
    print(f"일반 객관식: {regular_correct}/{regular_total} ({regular_acc:.1%})")

# 상세 로그 저장
save_detailed_logs(pred_long, "evaluation_ox")

logger.info("=" * 60)
logger.info("O, X 문제를 포함한 평가 완료")
logger.info("=" * 60)


In [ ]:
# -----------------------------
# 7) 추가 유틸리티 함수들
# -----------------------------

def print_evaluation_summary(acc_df: pd.DataFrame, pred_long_df: pd.DataFrame):
    """평가 결과 요약을 보기 좋게 출력"""
    print("\n" + "="*80)
    print("📊 평가 결과 상세 요약")
    print("="*80)
    
    # 기본 통계
    total_predictions = len(pred_long_df)
    valid_predictions = len(pred_long_df.dropna(subset=['answer']))
    invalid_predictions = total_predictions - valid_predictions
    
    print(f"📈 전체 예측 수: {total_predictions:,}")
    print(f"✅ 유효 예측: {valid_predictions:,} ({valid_predictions/total_predictions*100:.1f}%)")
    print(f"❌ 무효 예측: {invalid_predictions:,} ({invalid_predictions/total_predictions*100:.1f}%)")
    
    # 모델별 정확도
    print(f"\n🏆 모델별 정확도 순위:")
    for i, (_, row) in enumerate(acc_df.iterrows(), 1):
        accuracy = row['accuracy']
        if pd.isna(accuracy):
            print(f"  {i}. {row['model_name']}: N/A (데이터 없음)")
        else:
            print(f"  {i}. {row['model_name']}: {accuracy:.3f} ({accuracy*100:.1f}%)")
    
    print("="*80)

def save_detailed_logs(pred_long_df: pd.DataFrame, filename_prefix: str = "evaluation"):
    import datetime as dt
    """상세한 로그를 CSV로 저장"""
    timestamp = dt.datetime.now().strftime("%Y-%m-%d_%H%M%S")
    
    # 예측 결과 상세 로그
    pred_log_filename = f"{filename_prefix}_predictions_{timestamp}.csv"
    pred_long_df.to_csv(pred_log_filename, index=False, encoding='utf-8-sig')
    logger.info(f"상세 예측 로그 저장: {pred_log_filename}")
    
    # 모델별 통계
    model_stats = pred_long_df.groupby('model_name').agg({
        'answer': ['count', lambda x: x.notna().sum(), lambda x: x.isna().sum()]
    }).round(3)
    model_stats.columns = ['총_예측수', '유효_예측수', '무효_예측수']
    model_stats['유효율'] = (model_stats['유효_예측수'] / model_stats['총_예측수'] * 100).round(1)
    
    stats_filename = f"{filename_prefix}_model_stats_{timestamp}.csv"
    model_stats.to_csv(stats_filename, encoding='utf-8-sig')
    logger.info(f"모델 통계 저장: {stats_filename}")

def check_data_quality(df_all: pd.DataFrame, df_sample: pd.DataFrame):
    """데이터 품질 검사"""
    logger.info("데이터 품질 검사 시작...")
    
    issues = []
    
    # 1. 빈 문제 검사
    empty_questions = df_all[df_all['question'].str.strip() == '']
    if len(empty_questions) > 0:
        issues.append(f"빈 문제: {len(empty_questions)}개")
    
    # 2. 빈 선지 검사
    empty_options = df_all[(df_all['opt1'].str.strip() == '') & 
                          (df_all['opt2'].str.strip() == '') & 
                          (df_all['opt3'].str.strip() == '') & 
                          (df_all['opt4'].str.strip() == '') & 
                          (df_all['opt5'].str.strip() == '')]
    if len(empty_options) > 0:
        issues.append(f"빈 선지 문제: {len(empty_options)}개")
    
    # 3. 정답 없는 문제 검사
    no_answer = df_all[df_all['answer_set'].apply(len) == 0]
    if len(no_answer) > 0:
        issues.append(f"정답 없는 문제: {len(no_answer)}개")
    
    # 4. 중복 문제 검사
    duplicates = df_all[df_all.duplicated(subset=['question'], keep=False)]
    if len(duplicates) > 0:
        issues.append(f"중복 문제: {len(duplicates)}개")
    
    if issues:
        logger.warning("데이터 품질 이슈 발견:")
        for issue in issues:
            logger.warning(f"  - {issue}")
    else:
        logger.info("데이터 품질 검사 통과 ✅")
    
    return issues


In [ ]:
# -----------------------------
# 8) 사용 예시 (개선된 버전)
# -----------------------------
# 데이터 준비
data = random_multiple
models = ['anthropic/claude-sonnet-4.5']
# , 'google/gemini-2.5-flash', 'openai/gpt-5']  # 실제 모델명

# 평가 실행 (진행상황과 로깅이 포함된 버전)
logger.info("=" * 50)
logger.info("평가 시작")
logger.info("=" * 50)

# 데이터 품질 검사
df_all_temp = json_to_df_all(data)
quality_issues = check_data_quality(df_all_temp, df_all_temp.sample(n=min(50, len(df_all_temp)), random_state=42))

# 평가 실행
df_all, pred_long, pred_wide, acc = run_eval_pipeline(
    data, models, sample_size=50, batch_size=10, seed=42, mock_mode=False
)

# 상세 결과 출력
print_evaluation_summary(acc, pred_long)

# 상세 로그 저장
save_detailed_logs(pred_long, "evaluation")

logger.info("=" * 50)
logger.info("평가 완료")
logger.info("=" * 50)

In [ ]:
save_detailed_logs(pred_long, "evaluation")

In [ ]:
# 결과 저장 (개선된 버전)
import datetime as dt

# 파일명에 타임스탬프 추가
timestamp = dt.datetime.now().strftime("%Y-%m-%d_%H%M%S")
filename = f"evaluation_results_{timestamp}.xlsx"

logger.info(f"결과를 {filename}에 저장 중...")

try:
    with pd.ExcelWriter(filename, engine="openpyxl") as w:
        df_all.to_excel(w, index=False, sheet_name="전체데이터")   # 통합 뷰
        pred_wide.to_excel(w, index=False, sheet_name="모델별예측")         # 모델별 예측(가로)
        acc.to_excel(w, index=False, sheet_name="정확도")       # 모델별 정확도 요약
        
    logger.info(f"결과 저장 완료: {filename}")
    
    # 결과 요약 출력 (이미 print_evaluation_summary에서 처리됨)
    print(f"\n📁 결과 파일 저장 완료: {filename}")
    
except Exception as e:
    logger.error(f"결과 저장 실패: {str(e)}")
    print(f"결과 저장 중 오류 발생: {str(e)}")

In [ ]:
# # -----------------------------
# # 9) 테스트 및 검증
# # -----------------------------

# # Mock 모드로 빠른 테스트
# print("🧪 Mock 모드로 빠른 테스트 실행...")
# test_data = random_multiple[:10]  # 10개만 테스트
# test_models = ['test-model']

# try:
#     df_all_test, pred_long_test, pred_wide_test, acc_test = run_eval_pipeline(
#         test_data, test_models, sample_size=10, batch_size=5, seed=42, mock_mode=True
#     )
    
#     print("✅ Mock 테스트 성공!")
#     print(f"   - 처리된 문제 수: {len(df_all_test)}")
#     print(f"   - 예측 결과 수: {len(pred_long_test)}")
#     print(f"   - 정확도: {acc_test.iloc[0]['accuracy']:.3f}")
    
# except Exception as e:
#     print(f"❌ Mock 테스트 실패: {str(e)}")
#     logger.error(f"Mock 테스트 실패: {str(e)}")

# print("\n" + "="*50)
# print("🚀 실제 평가를 실행하려면 위의 Cell 8을 실행하세요!")
# print("="*50)
